In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
directory = "../input/home-depot-product-search-relevance"
processed_directory = "../input/d/cshige/home-depot-dts/"

# **Khai báo các thư viện cần thiết**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *

from gensim.utils import simple_preprocess, tokenize
from gensim.models import KeyedVectors

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import keras as kr
import matplotlib
import xgboost
import gensim
import spacy

from time import time
import pickle
import math
import re

# **Khám phá dữ liệu**

## Dữ liệu train

In [ ]:
train = pd.read_csv(directory + "/train.csv.zip", encoding = "ISO-8859-1", index_col= "id")

In [ ]:
train.info()

Qua cái nhìn tổng quan của dữ liệu train, ta thấy có 5 cột, dựa vào tên ta có thể suy ra được:

- product_uid: Mã của sản phẩm. Các sản phẩm khác nhau sẽ có mã sản phẩm khác nhau.
- product_title: Tiêu đề tên sản phẩm.
- search_term: Từ khóa tìm kiếm.
- relevance: Mức tương quan giữa từ khóa tìm kiếm và tiêu đề sản phẩm.

Ta thấy có 74067 dữ liệu, hoàn toàn không có dữ liệu null, nên ta sẽ không phải xử lý dữ liệu null trong bảng.

Tuy nhiên, product_title và search_term mang loại object, ta sẽ cần xem chi tiết hơn về chúng.

Cell dưới đây sẽ cho ta thấy một vài dòng đầu của dữu liệu train.

In [ ]:
train.head(5)

Ta nhận thấy cột tiêu đề sản phảm chứa các dữ liệu văn bản chưa được chuẩn hóa: chứa các ký tự in hoa lẫn thường, ký tự đặc biệt, ... ta sẽ cần chuẩn hóa lại để tiện cho việc xử lý sau này.

In [ ]:
train['relevance'].describe()

Phần lớn độ tương quan nằm trong khoảng từ 2.00 đến 3.00 (50% là 2.33).\
Cell dưới đây sẽ cho ta biểu đồ trực quan cho kết luận trên.

In [ ]:
distribution = train['relevance'].value_counts().sort_index()
plt.figure(figsize=(6, 4))
plt.ylabel('tần suất')
plt.xlabel('giá trị tương quan')
distribution.plot(kind = 'bar')
plt.show()

Ta thử xem những dữ liệu có tương quan cao nhất

In [ ]:
hight_relevance = train.loc[train["relevance"] == 3]
hight_relevance[['product_title', 'search_term']].head()

## Dữ liệu test

In [ ]:
test = pd.read_csv(directory + "/test.csv.zip", encoding = "ISO-8859-1", index_col= "id")

In [ ]:
test.info()

In [ ]:
test.head(5)

Tổng quan dữ liệu test:
- 166693 dữ liệu, lớn gấp hơn 2 lần dữ liệu train
- Không bao gồm trường relevant

Sau khi tìm hiểu dữ liệu train và test, ta sẽ cùng nhìn rõ hơn về trường search_term ở cả 2 bảng.

In [ ]:
distribution_train = train.search_term.str.split().apply(len).value_counts().sort_index()
distribution_test = test.search_term.str.split().apply(len).value_counts().sort_index()
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Lượng từ khóa mỗi lần tìm kiếm')

distribution_train.plot(kind = 'bar', sharex = 1, figsize = (7,5), ax = ax1, title = 'train')
distribution_test.plot(kind = 'bar', colormap = 'jet', figsize = (7, 5), ax = ax2, title = 'test')

Qua bảng trên, ta thấy phần lớn đều trong khoảng từ 2 đến 4 từ, và có rất ít dữ liệu có quá nhiều từ.\
Từ đó cần phải xử lý triệt để những từ ngữ không cần thiết, làm loãng dữ liệu và gây ảnh hưởng đến trường giá trị tương quan.

## Dữ liệu thuộc tính

In [ ]:
attributes=pd.read_csv(directory + "/attributes.csv.zip")

In [ ]:
attributes.info()

In [ ]:
attributes.head(5)

Qua những thôn tin tổng quan về dữ liệu thuộc tính, ta có một số nhận xét sau:

- product_uid: Mã sản phẩm. Mã này lại được ghi dưới dạng số thập phân, ta sẽ cần sửa lại thành dạng số nguyên để đồng nhất với các dữ liệu khác.
- name: Tên thuộc tính. Mỗi sản phẩm theo product_uid có thể mang nhiều thuộc tính. Câu hỏi đặt ra ở đây là tại sao cả 5 dòng đều có name chứa 'Bullet'. Thông tin này hiện tại không mang ý nghĩa. Ta sẽ phân tích ở cell dưới.
- value: Thông tin về thuộc tính tương ứng. Trường thông tin này mang nhiều loại ký tự.

In [ ]:
attributes['name'].unique()

Như vậy trường name có nhiều tên thuộc tính là Bullet, tức là các gạch đầu dòng, không mang ý nghĩa gì. Tuy nhiên, lại có những thông tin mang ngữ nghĩa.

In [ ]:
len(attributes['name'].unique())


Có 5411 trường thông tin khác nhau, trên tổng số 2 triệu mẫu, tức là ta hoàn toàn có khả năng dựa vào thông tin này để lấy đặc trưng.

In [ ]:
attribute_counts = attributes.name.value_counts()
print(attribute_counts)

Có một trường thông tin là MFG Brand Name xuất hiện nhiều nhất, là trường mang thông tin nhà sản xuất sản phẩm đó (Manufacture Brand Name).

In [ ]:
attributes[attributes.name == "MFG Brand Name"]

Bằng việc lấy ra những hàng có trường name là "MFG Brand Name", ta thấy có 86250 hàng là có tên nhãn hiệu.\
Còn lại, những hàng mà có trường name chứa "Bullet" thì trường value không có giá trị trong bài toán này.

Nếu ta nhìn sâu vào dữ liệu thuộc tính thêm nữa thì ngoài nhãn hiệu, ta cũng có một loại giá trị đó là cấu tạo "Material".\
Nhãn hiệu và cấu tạo ở đây chính là 2 điểm mấu chốt để ta tiến hành xử lý dữu liệu sau này.\
Cell dưới đây là minh họa cho trường có giá trị cấu tạo.

In [ ]:
attributes[attributes.name == "Material"]

## Dữ liệu mô tả

In [ ]:
product_descriptions = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv.zip')

In [ ]:
product_descriptions.info()

In [ ]:
product_descriptions.head()

Nhìn vào dữ liệu, ta thế dữ liệu này mang 2 thông tin:

- product_uid: Mã sản phẩm.  
- product_description: Dữ liệu văn bản mô tả sản phẩm. Dữ liệu này bao gồm cả các ký tự latin viết hoa, viết thường, các chữ số và ký tự đặc biệt.

Ta thấy bảng mô tả này cũng mang nhiều thông tin quan trọng, nhưng hiện tại lại rời rạc với các dữ liệu khác.\
Vì vậy, ta sẽ nối nó với dữ liệu train và test.

In [ ]:
train = pd.merge(train, product_descriptions, on='product_uid')
test = pd.merge(test, product_descriptions, on='product_uid')

In [ ]:
train.head()

Tương tự như trên, ta cùng khám phá lượng từ khóa mô tả cho cả dữ liệu train và test.

In [ ]:
distribution_train = train.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
distribution_test = test.product_description.str.split().apply(len).divmod(10)[0].value_counts().nlargest(40).sort_index()
fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle('Lượng từ khóa mô tả của mỗi sản phẩm')

distribution_train.plot(kind = 'bar', sharex = 1, figsize = (10,5), ax = ax1, title = 'train')
distribution_test.plot(kind = 'bar', colormap = 'jet', figsize = (10, 5), ax = ax2, title = 'test')

Lượng từ phổ biến nằm trong khoảng từ 4 đến 14 từ, như vậy trường này cũng có nhiều từ gây nhiễu, loãng dữ liệu, cần xử lý để mô hình cho kết quả tốt nhất.

# **Xử lý dữ liệu**

Như đã đề cập ở trên thì dữ liệu thuộc tính có trường name thì chỉ có các dòng mang giá trị "MFG Brand Name" và "Material" là quan trọng.\
Vì vậy ta sẽ lấy riêng ra 1 bảng những dòng có giá trị đó và nối vào dữ liệu train và test.

In [ ]:
brand_attributes = attributes[attributes.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
material_attributes = attributes[attributes.name == "Material"][["product_uid", "value"]].rename(columns={"value": "material"})
brand_attributes.drop_duplicates(subset=['product_uid'], inplace = True)
material_attributes.drop_duplicates(subset=['product_uid'], inplace = True)

Để nhất quán và tránh lỗi lặp cột thì ta sẽ đổi tên cột mà có giá trị "MFG Brand Name" thành cột "brand", còn "Material" thì đổi thành cột "material".\
Đồng thời loại bỏ những dòng dữ liệu trùng nhau

Sau đó ta nối dữ liệu vừa có vào dữ liệu train và test. Những dòng nào bị null thì ta sẽ để là xâu rỗng cho an toàn.

In [ ]:
train = pd.merge(train, brand_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, brand_attributes, on='product_uid',  how='left').fillna('')

train = pd.merge(train, material_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, material_attributes, on='product_uid',  how='left').fillna('')

Xem lại bảng train và test

In [ ]:
train.head()

In [ ]:
test.head()

Như vậy là đã xong công đoạn ghép nối những dữ liệu bảng rời rạc ban đầu thành một dữ liệu thống nhất đầy đủ.

Bước tiếp theo ta sẽ tiến hành xử lý văn bản để có được một dữ liệu sạch, làm tiền đề để huấn luyện mô hình.

Code stemming tham khảo tại https://gist.github.com/susanli2016/b83d148de7394821509bd5172d2c96d3

In [ ]:
strNum = {'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}
def str_stem(s): 
    if isinstance(s, str):
        # Tách cụm bị dấu chấm ngắt giữa thành các từ
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s)
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        
        # Đổi đơn vị đo thành từ chuẩn quốc tế
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. ", s)
        s = re.sub(r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. ", s)
        s = re.sub(r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. ", s)
        s = re.sub(r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. ", s)
        s = re.sub(r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", r"\1 gal. per min. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr ", s)
        
        # Xóa các ký tự đặc biệt
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("&nbsp;"," ")
        s = s.replace("&amp;","&")
        s = s.replace("&#39;","'")
        s = s.replace("/>/Agt/>","")
        s = s.replace("</a<gt/","")
        s = s.replace("gt/>","")
        s = s.replace("/>","")
        s = s.replace("<br","")
        s = s.replace("<.+?>","")
        s = s.replace("[ &<>)(_,;:!?\+^~@#\$]+"," ")
        s = s.replace("'s\\b","")
        s = s.replace("[']+","")
        s = s.replace("[\"]+","")
        s = s.replace("-"," ")
        s = s.replace("+"," ")
        s = s.replace("..",".")
        s = s.replace(",","")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("/"," ")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = s.replace("  "," ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")

        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = " ".join([re.sub('[^A-Za-z0-9-./]', ' ', word) for word in s.lower().split()])
        return s
    else:
        return "null"

Thư viện kiểm tra chính tả

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

Loại bỏ stopword, sau đó chuyển những danh/động từ về dạng nguyên gốc của từ đó, rồi tokenize câu.

In [ ]:
stop_words = set(stopwords.words('english')) 
wordnet_lemmatizer = WordNetLemmatizer()
gensim.utils.tokenize
def preprocessing(doc, correction = 0):
    words = []
    doc  = str_stem(doc)
    for word in doc.split(' '):
        if word not in stop_words:
            if correction == 1:
                if len(spell.unknown([word])):
                    word = spell.correction(word)
            word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
            word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
            word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
            words.append(word3)
    return ' '.join(words)

Cell tiếp theo ta tiến hành xử lý văn bản trong tập train và test.\
Sau đó lưu lại tập train và test ra file để tiện dùng cho lần sau.

In [ ]:
%%time

train.search_term = [preprocessing(i, correction = 1) for i in train.search_term]
train.product_description = [preprocessing(i) for i in train.product_description]
train.brand = [preprocessing(i) for i in train.brand]
train.material = [preprocessing(i) for i in train.material]
train.product_title = [preprocessing(i) for i in train.product_title]

test.search_term = [preprocessing(i, correction = 1) for i in test.search_term]
test.product_description = [preprocessing(i) for i in test.product_description]
test.brand = [preprocessing(i) for i in test.brand]
test.material = [preprocessing(i) for i in test.material]
test.product_title = [preprocessing(i) for i in test.product_title]

train.to_pickle("proccessed_train")
test.to_pickle("proccessed_test")

'''train = pd.read_pickle(processed_directory + "proccessed_train")
test = pd.read_pickle(processed_directory + "proccessed_test")'''

# Xử lý, trích chọn đặc trưng

### Vì đây là bài toán độ tương quan tìm kiếm (search relevant) nên ta sẽ tiến hành trích chọn những đặc trưng giữa **từ khóa tìm kiếm**(search_term) và các thông tin khác.

Trước tiên ta ghép nối dữ liệu train và test lại để thuận tiện cho việc xử lý.\
Cùng với đó, không phải dòng sản phẩm nào cũng đủ thông tin về "brand" và "material" như đã xử lý ở trên, ta để các ô dữ liệu bị null thành xâu rỗng.

In [ ]:
features = pd.concat((train, test), axis=0, ignore_index=True)
print(str(len(train)) + ' ' + str(len(test)) + ' ' + str(len(features)))
del train
del test

features['brand'] = features['brand'].replace(np.nan, '', regex=True)
features['material'] = features['material'].replace(np.nan, '', regex=True)

### Đặc trưng không gian vector (Word embedding)

In [ ]:
from nltk.corpus import brown
embed_model = gensim.models.Word2Vec(brown.sents())
embed_model.save('brown.embedding')
model = gensim.models.Word2Vec.load('brown.embedding')

Ta sẽ tính trung bình tất cả các số thể hiện sự giống nhau giữa từng từ của "từ khóa tìm kiếm" và từng từ của mô tả sản phẩm.

In [ ]:
%%time
def embeding_similarity_calculator(s, t, i):
    _sum = 0
    avg = 0
    if len(s.split()) == 0 :
        return 0
    for s_word in s.split():
        _max = 0
        for t_word in t.split():
            if ((s_word in model.wv) and (t_word in model.wv)):
                _max = max(_max, model.wv.similarity(s_word, t_word))
        _sum += _max
    avg = _sum/ len(s.split())
    return avg
features["word_ebed_similarity"] = [embeding_similarity_calculator(features["search_term"][i], features["product_description"][i], i) for i in range(0, len(features))]

### Tiếp theo ta tính đặc trưng TF-IDF 

Ta cũng tính toán giữa 2 cột từ khóa tìm kiếm và mô tả sản phẩm, rồi lưu lại để thuận tiện cho các lần chạy tiếp theo.

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='char_wb', ngram_range = (3,3), max_features = 1500)
tfidf_des = tfidf_vect.fit_transform(features.product_description).toarray()
tfidf_search = tfidf_vect.transform(features.search_term).toarray()

outfile = open("tfidf_des",'wb+')
pickle.dump(tfidf_des, outfile)
outfile = open("tfidf_search",'wb+')
pickle.dump(tfidf_search, outfile)

'''infile = open(processed_directory + "tfidf_des",'rb')
tfidf_des = pickle.load(infile)
infile = open(processed_directory + "tfidf_search",'rb')
tfidf_search = pickle.load(infile)'''

Ta sử dụng cosin để tính sự giống nhau.

In [ ]:
from scipy.spatial import distance
features["tfidf_cosine_distance"] = [distance.cosine(tfidf_search[i], tfidf_des[i]) for i in range(0, len(tfidf_des))]

outfile = open("features",'wb')
pickle.dump(features, outfile)

'''infile = open(processed_directory + "features_18_4",'rb')
features = pickle.load(infile)'''

# **Mô hình**

Sau khi có đủ dữ liệu đặc trưng, ta loại bỏ những cột dữ liệu dạng text, chỉ giữ lại những cột dạng số để train.

In [ ]:
features = pd.DataFrame(features).fillna(0)
df_train = features.iloc[:74067]
df_test = features.iloc[74067:]
y_train = df_train['relevance']
df_train = df_train.drop(columns=['product_title','product_description','brand','material','search_term','relevance'])
df_test = df_test.drop(columns=['product_title','product_description','brand','material','search_term', 'relevance'])
df_train = df_train.sort_values('product_uid')

Ta sẽ dùng 2 tham số là Lỗi trung bình tuyệt đối (mean absolute error-mae) và Lỗi trung bình bình phương (mean squared error-mse) để đánh giá mô hình. 
- mae là giá trị trung bình trên tập xác nhận validation của các giá trị tuyệt đối của sự khác biệt giữa tập dự đoán predict và tập train.
- mse là sự khác biệt giữa các giá trị trên tập dự đoán và các giá trị tập train tương ứng được bình phương và sau đó được tính trung bình trên mẫu.

Để thuận tiện, ta sẽ gộp chung bước train và lấy tham số vào chung 1 hàm.

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def modelEvaluate(model, X_train, y_train, x_val, y_val, label_encode = None):
    model.fit(X_train, y_train)
    pred = model.predict(x_val)
    _pred = pred
    _y_val = y_val
    if label_encode is not None:
        _y_val = label_encode.inverse_transform(_y_val)
        _pred = label_encode.inverse_transform(_pred)
    mae, mse = (mean_absolute_error(_pred, _y_val), mean_squared_error(_pred, _y_val))
    return (mae, mse)

Sau khi xây dựng xong hàm tính tham số đánh giá, ta xây dựng hàm để kiểm chứng chéo (cross validate).

Ta sử dụng hàm KFold để phân chia tập train và test thành k cụm liên tục và với mỗi cụm thì ta sẽ kiểm chứng chéo với k-1 cụm còn lại.

Với mỗi cụm, ta tính tham số mae, mse rồi lấy trung bình tổng, trung bình bình phương của mô hình.

In [ ]:
from sklearn.model_selection import KFold

def crossValidate(model, label_encoder = None):
    mae_score = []
    mse_score = []
    kf = KFold(n_splits=4)
    kf.get_n_splits(df_train)
    for train_index, test_index in kf.split(df_train):
        X, X_test = df_train.iloc[train_index], df_train.iloc[test_index]
        y, y_test = y_train.iloc[train_index], y_train.iloc[test_index]
        (mae, mse) = modelEvaluate(model, X, y, X_test, y_test, label_encode = label_encoder)
        mae_score.append(mae)
        mse_score.append(mse)
    return [sum(mae_score)/len(mae_score), math.sqrt(sum(mse_score)/len(mse_score))]

### Hồi quy

Ta sẽ tiến hành huấn luyện và so sánh kết quả trên một vài mô hình hồi quy khác nhau.

Ở đây ta sẽ dùng một vài mô hình phổ biến như RandomForest, GradientBoosting, Bayes ...

In [ ]:
%%time
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import linear_model

result = []
result.append(('RandomForest',
               crossValidate(RandomForestRegressor(n_estimators=700,
                                                   max_depth=6,
                                                   random_state=42))))
result.append(('XGBoost',
               crossValidate(XGBRegressor(colsample_bytree=0.4,
                                          learning_rate=0.1,
                                          max_depth=6,
                                          n_estimators=700,
                                          reg_alpha=0.075,
                                          reg_lambda=0.045,
                                          subsample=0.6,
                                          seed=42))))
result.append(('GradientBoosting',
               crossValidate(GradientBoostingRegressor(n_estimators=700,
                                                       max_depth=6,
                                                       random_state=42))))

In [ ]:
result

Sau khi có kết quả các tham số mae, mse mà các mô hình đưa ra, ta tiếp tục stack các mô hình vào một mô hình cuối cùng để tìm kết quả tốt nhất.

Việc stack các mô hình đã train, cùng với mô hình StackingRegressor sẽ tối ưu vì nó fit cùng đặc điểm hồi quy với các mô hình trên. Ta sẽ thu được một mô hình là hội tụ các đặc điểm của tất cả các mô hình còn lại.

In [ ]:
%%time
from sklearn import linear_model
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import BayesianRidge
estimators = [('RandomForest', 
               RandomForestRegressor(n_estimators=700,
                                     max_depth=6,
                                     random_state=42)),
              ('XGBoost',
               XGBRegressor(colsample_bytree=0.4,
                            learning_rate=0.1,
                            max_depth=6,
                            n_estimators=700,
                            reg_alpha=0.075,
                            reg_lambda=0.045,
                            subsample=0.6,
                            seed=42))]
stacked_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=BayesianRidge())
result.append(('StackedBayesianRidge', crossValidate(stacked_reg)))

Sau khi train model cuối, ta cùng nhau biểu đồ hóa tham số mất mát xem model nào cho kết quả tốt nhất.

Cell dưới đây sẽ làm nhiệm vụ này.

In [ ]:
labels = []
mae = []
rmse = []
for _name, _score in result:
    labels.append(_name)
    mae.append(_score[0])
    rmse.append(_score[1])

#Plotting
x = np.arange(len(labels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots(figsize=(15,6))
rects1 = ax.bar(x - width/2, mae, width, label='MAE')
rects2 = ax.bar(x + width/2, rmse, width, label='RMSE')

ax.set_ylabel('Scores')
ax.set_title('Scores của từng models')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

Đúng như dự đoán, mô hình StackedBayesianRidge cho kết quả tốt nhất với dộ mất mất thấp nhất như biểu đồ.

# Dự đoán và xuất kết quả

In [ ]:
y_train = features.iloc[:74067]['relevance']
stacked_reg.fit(df_train, y_train)
pred = stacked_reg.predict(df_test)

submission = pd.read_csv(directory + "/sample_submission.csv.zip")
pred = [min(3, i) for i in pred]
pred = [max(1, i) for i in pred]
submission['relevance'] = pred
submission.to_csv('submission.csv', index=False)